In [3]:
ROOT = "data"

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.readers.file import PyMuPDFReader

In [15]:
import logging
for name in ("httpx", "httpcore", "llama_index", "llama_index.core"):
    logging.getLogger(name).setLevel(logging.WARNING)  # ou ERROR

In [ ]:
# Ler TODOS os PDFs (subpastas incluídas) usando PyMuPDF
docs = SimpleDirectoryReader(
    ROOT, 
    required_exts=[".pdf"],
    recursive=True,
    file_extractor={".pdf": PyMuPDFReader()}  # força extração de texto
).load_data()

In [19]:
from pathlib import PurePosixPath

for d in docs:
    p = PurePosixPath((d.metadata.get("file_path") or d.metadata.get("filename") or "").replace("\\","/"))
    parts = [s.lower() for s in p.parts]
    d.metadata["org"] = "ANVISA" if "anvisa" in parts else ("MS" if "ms" in parts else "OUTRO")
    d.metadata["doctype"] = ("LEI" if "leis" in parts else
                             "RDC" if "rdc" in parts else
                             "PORTARIA" if "portarias" in parts else
                             "IN" if "in" in parts else
                             "RE" if "re" in parts else "OUTRO")

In [21]:
# Chunking “largo” para normas (completude)
nodes = SentenceSplitter(
    chunk_size=1400, chunk_overlap=200, 
    paragraph_separator="\n\n"
).get_nodes_from_documents(docs)


In [22]:
# Índice único (embeddings locais via Ollama)
index = VectorStoreIndex(nodes, 
                         embed_model=OllamaEmbedding("nomic-embed-text"))

In [ ]:
# Query Engine (PT-BR, top_k ligeiramente maior + síntese que costura)
llm = Ollama(
    model="llama3:8B",
    temperature=0,
    system_prompt="Responda SEMPRE em português do Brasil; use apenas os trechos recuperados; se faltar evidência, diga 'Não encontrei'."
)

qe = index.as_query_engine(
    llm=llm,
    similarity_top_k=8,      # mais contexto p/ perguntas amplas
    response_mode="refine",  # junta informações de vários trechos
)

In [27]:
# Duas perguntas: uma ampla e uma específica (demonstra o índice único)
q1 = "Quais dispositivos tratam de participação social no SUS?"
q2 = "Quais são os princípios do SUS na Lei 8.080/1990?"

In [ ]:
resp1 = qe.query(q1)
print("\nRESPOSTA:\n", resp1)


RESPOSTA:
 Os dispositivos que tratam de participação social no Sistema Único de Saúde (SUS) incluem disposições que estabelecem a promoção e proteção da saúde dos trabaladores, assim como visa à recuperação e reabilitação da saúde dos trabaladores submetidos aos riscos e agravos advindos das condições de trabalho.


In [17]:
resp2 = qe.query(q2)
print("\nRESPOSTA:\n", resp2)


RESPOSTA:
 Os princípios do Sistema Único de Saúde (SUS) são definidos pela Lei Nº 8.080, de 19 de setembro de 1990, que estabelece a promoção, proteção e recuperação da saúde como objetivos fundamentais.


In [28]:

for q in (q1, q2):
    print("\n" + "="*80)
    print("PERGUNTA:", q)
    resp = qe.query(q)
    print("\nRESPOSTA:\n", getattr(resp, "text", None) or getattr(resp, "response", ""))
    print("\nFONTES:")
    for i, s in enumerate(getattr(resp, "source_nodes", []) or [], 1):
        meta = getattr(s.node, "metadata", {})
        print(f"- {i}.", meta.get("file_path") or meta.get("filename") or "(sem nome)")


PERGUNTA: Quais dispositivos tratam de participação social no SUS?

RESPOSTA:
 Não encontrei

FONTES:
- 1. c:\Users\ander\Documents\Projetos\Aulas\proadi-sus-ciencia-de-dados-ia\aulas\Aulas\DL - Agentes\DL - Rag\data\anvisa\in\2018\in_28_julho_2018.pdf
- 2. c:\Users\ander\Documents\Projetos\Aulas\proadi-sus-ciencia-de-dados-ia\aulas\Aulas\DL - Agentes\DL - Rag\data\lei-8080.pdf
- 3. c:\Users\ander\Documents\Projetos\Aulas\proadi-sus-ciencia-de-dados-ia\aulas\Aulas\DL - Agentes\DL - Rag\data\anvisa\rdc\2018\rdc_243_julho_2018.pdf
- 4. c:\Users\ander\Documents\Projetos\Aulas\proadi-sus-ciencia-de-dados-ia\aulas\Aulas\DL - Agentes\DL - Rag\data\ms\leis\lei_8080_setembro_1990.pdf
- 5. c:\Users\ander\Documents\Projetos\Aulas\proadi-sus-ciencia-de-dados-ia\aulas\Aulas\DL - Agentes\DL - Rag\data\lei-8080.pdf
- 6. c:\Users\ander\Documents\Projetos\Aulas\proadi-sus-ciencia-de-dados-ia\aulas\Aulas\DL - Agentes\DL - Rag\data\lei-8080.pdf
- 7. c:\Users\ander\Documents\Projetos\Aulas\proadi-sus-ci